## Vehicle Detection Project 

The goals / steps of this project are the following:

* Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier
* Optionally, you can also apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector. 
* Note: for those first two steps don't forget to normalize your features and randomize a selection for training and testing.
* Implement a sliding-window technique and use your trained classifier to search for vehicles in images.
* Run your pipeline on a video stream (start with the test_video.mp4 and later implement on full project_video.mp4) and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
* Estimate a bounding box for vehicles detected.


Here I will consider the [Rubric](https://review.udacity.com/#!/rubrics/513/view) points individually and describe how I addressed each point in my implementation.  

### Pipeline of project:

This project can be divided into two parts in general: training classifier and processing video 

Training part will include the following procedures:
   * Import data
   * Feature extraction
   * Training and testing
   
Processing video part includes the following steps:
   * Features extraction for slide windows 
   * window feature prediction
   * Outline detected vehicle

### 1. Import data and example display

`Training_data.py` is used for setting key parameters for the whole project and generating a single instance-label data set. Since I need to try several combinitions of color space, orient number of hog features, for different sets parameters, this file will generate a `dataset.p` and a `parameter.p` to store those information. Since parameters will also be used in following procedure, call from a saved file will not make any mistake as long as these parameters are all set here.

To extract the features, I called `feature_extraction()` function from `feature_extraction.py`. This function will generate a feature vector from input data. According to parameters, this function will determine whether to include spatial features, color histogram features or hog features in to the feature vector or not. 

The final dataset has 17760 instances and each instances has 8460 features.

##### * dataset information

    The number of cars is  8792

    The number of notcars is  8968

    feature dimension is:  (17760, 8460)

    label dimension is:  (17760,)


##### * parameter information

    color_space: YCrCb  pixels per cell:  8  orient number for hog: 9

    cells per block: 2  hog channels: ALL  spatial histogram: (32, 32)  histogram bins: 32

    spatial_feat:  True  hist_feat:  True  hog_feat:  True


##### * feature informations
    
    Spatial features dimension: 3072. Spatial features dimension: 96.

    Spatial features dimension: 5292. Spatial features dimension: (8792, 8460).

    time for feature extraction of cars and noncars are:  48.35s, 95.31s


#### Histogram of Oriented Gradients (HOG)

I created a function called `get_hog_features()` in `feature_extraction.py` in lines 25 through 36 to utilize hog() function in skimage library to get HOG features. This function can only extract HOG features for a single channel. I explored different color spaces and different parameters (`orientations`, `pixels_per_cell`, and `cells_per_block`).  Here is an example using the `YCrCb` color space and HOG parameters of `orientations=9`, `pixels_per_cell=(8, 8)` and `cells_per_block=(2, 2)`Three channels' hog images of randomly selected car and notcar is shown below

![alt text](./test_results/hog_features.png)

#### Feature combination selection

After experiments and searching for practical experience online, I decide to combine spatial features, color histogram features and all three channels' hog features in to the feature vector which makes its length is 8460. Though very long, it can provide enough information for training.

### 2. Train the classifier

To train the classifier, I used another file called `classifier.py`. This file has three functions: generating training and test set, training classifier, and test classifier.

#### Training set and test set
The training set takes 80% data (14208 instances) from original dataset, and test set is 20%. Using `train_test_split()` from sklearn library, training and test data are all randomly selected. 

#### classifier choice

We learned decision tree, SVM and Naive Bayes from class, I choose to use SVM as the classifier. There are two different classifiers in sklearn library: LinearSVC and SVC. I tried them both. It turned out that LinearSVC is very fast and classifier it trained has less paramers. To training 14208 instances, it only uses about around 30s and acquire 98% accuracy. For SVC, it will take more than 3200s (nearly an hour) to train, though can have 99.5% accuracy. Finally I chose LinearSVC



### 3. Sliding Window Search

There are two ways to generate features of sliding windows of test frames. First is to define window positions with different dimensions and generate spatial features, color histogram features, and hog features for each window. Second one is to calculate hog features for the whole frame ones, then just generate spatial and color histogram features for each window. The second method is much faster than the first one, espetially when the number of windows increases. I create a function to do sliding window search and the function returns image with detected boxes and position array of those detected boxes. The code locates in `slide_window.py`.




### 4. Process False Positive and Overlapping Boxes.

I recorded the positions of positive detections in each frame of the video.  From the positive detections I created a heatmap and then thresholded that map to identify vehicle positions.  I then used `scipy.ndimage.measurements.label()` to identify individual blobs in the heatmap.  I then assumed each blob corresponded to a vehicle.  I constructed bounding boxes to cover the area of each blob detected. By setting the threshold for avoid false positive and overlapping.  

The image process code is in `image_process.py`

#### Here are six test images and their corresponding heatmaps

![alt text](./test_results/heat_123.png)
![alt text](./test_results/heat_456.png)

#### Here is the output after processing false positive and overlapping according to heatmaps
![alt text](./test_results/thresh_123.png)
![alt text](./test_results/thresh_456.png)


### Video Implementation

The video process code is in `video_pipeline.py`

In order to the box bounding cars more stable, I used a class to track boxes of last six frames and use all those box for tracking cars

Here's a [link to my video result](https://youtu.be/ehpgk5jWPIA)

### Discussion

#### 1. Briefly discuss any problems / issues you faced in your implementation of this project.  Where will your pipeline likely fail?  What could you do to make it more robust?

For the most important is to make sure parameters set at beginning will not change in the following procedure, since these paramters are used from data generation part, they will also be used for vehicle detection. To avoid change of parameters, save them to a .p file is a good choice.

Another thing is that when processing test image and video, we have to pay attention to the format of images and frames of video. Different formats will cause different values when we import those images. They have to be change to be the same as our training data. In our case, we need to divide 255 for all read in image and frame. 

From the video process, we can tell there are also cars on the left side which are going in the opposite direction, but in my pipeline these cars are not detected. May be they are too small. And for detect cars, when two cars are very close in fram, two boxes will integrate into a sigle big box, I think if I can keep it as separate would be better.